### Ontbrekende Gegevens

In [2]:
import pandas as pd
import numpy as np

Python's implementatie van `float` objecten bevat enkele speciale "getallen" om oneindigheid en "geen getal" voor te stellen (deze maken eigenlijk deel uit van de IEEE-specificatie voor floats, niet alleen van Python-specifieke zaken)

In [2]:
float('inf')

inf

Dit is eigenlijk een object van het type float:

In [3]:
type(float('inf'))

float

We kunnen dat `float` object ook uit de `math` module halen:

In [4]:
import math
math.inf

inf

Of zelfs vanuit de NumPy-bibliotheek:

In [5]:
np.inf

inf

Het hebben van deze "oneindig" float kan handig zijn wanneer je boven- en ondergrenzen moet definiëren.

De andere speciale float, en degene die we hier bekijken, is de `NaN` (geen getal) waarde - deze wordt in feite gebruikt om aan te geven dat een float getal ongedefinieerd is of niet representeerbaar.

Het wordt ook vaak gebruikt om ontbrekende gegevens in een array aan te geven.

In [6]:
float('NaN'), float('nan')

(nan, nan)

Het is ook beschikbaar in `math` en `numpy`:

In [7]:
math.nan, np.nan

(nan, nan)

**LET OP**: Probeer geen `NaN`-getal te vergelijken met een ander - ze zullen nooit gelijk zijn!

In [8]:
float('nan') == float('nan')

False

In [9]:
float('nan') is math.nan

False

En dit is eigenlijk wel logisch, als twee getallen niet gedefinieerd zijn, wie zegt dan dat ze gelijk zijn?

Hoe controleren we of een getal `NaN` is als we geen gelijkheidstests kunnen gebruiken?

De module `math` heeft de functie `isnan()` die we kunnen gebruiken:

In [10]:
math.isnan(float('NAN'))

True

In [11]:
math.isnan(np.nan)

True

De NumPy-module heeft die functie ook:

In [12]:
np.isnan(math.nan)

True

En als je je afvraagt waarom NumPy die functie gedefinieerd heeft, terwijl deze al in de `math` module zit, houd dan in gedachten dat NumPy functies universele functies zijn.

In [13]:
a = np.array([1, 2, np.nan, 3, np.nan])
a

array([ 1.,  2., nan,  3., nan])

In [14]:
np.isnan(a)

array([False, False,  True, False,  True])

#### Werken met Ontbrekende Waarden in Series

Laten we eens kijken naar `NaN` in de context van `Series` objecten:

In [15]:
s = pd.Series([3.14, 2.5, None, 5])
s

0    3.14
1    2.50
2     NaN
3    5.00
dtype: float64

Zoals je kunt zien, werd de array geïnterpreteerd als een float64-array, en de waarde `None` werd geconverteerd naar de `NaN` float:

In [16]:
type(s[2])

numpy.float64

Dus we kunnen `NaN`-waarden hebben voor floats, maar geldt dat ook voor integers?

Pandas is gebouwd op NumPy, en NumPy-arrays hebben niet het concept van `NaN` voor integers, dus Pandas ook niet.

Wat er uiteindelijk gebeurt, is dat Pandas de serie zal omzetten naar een float64 (of object) wanneer het een `None` of `NaN` tegenkomt:

In [17]:
pd.Series([1, 2, 3])

0    1
1    2
2    3
dtype: int64

In [18]:
pd.Series([1, 2, 3, None])

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [19]:
pd.Series([1, 2, 3, np.nan])

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

Dus voor numerieke types kunnen we zowel `NaN` als `None` gebruiken en zal Pandas indien nodig converteren naar het floating point `NaN`.

Maar we moeten voorzichtiger zijn met reeksen die van het type `object` zijn, zoals strings:

In [20]:
s = pd.Series(['a', 'b', None, np.nan])
s

0       a
1       b
2    None
3     NaN
dtype: object

Je zult hier zien dat `None` **niet** geconverteerd werd naar `NaN`.

Hoe testen we of een waarde in deze reeks "ontbreekt"?

In [21]:
s[2] is None

True

In [22]:
s[3] is None

False

Dus we kunnen geen test uitvoeren met `is None` voor beide gevallen, en ook kunnen we `isnan()` niet gebruiken:

In [23]:
try:
    math.isnan(s[2])
except TypeError as ex:
    print('TypeError:', ex)

TypeError: must be real number, not NoneType


Gelukkig biedt Pandas een paar functies die we kunnen gebruiken om om te gaan met ontbrekende waarden, gerepresenteerd door `None` of door `NaN`:
- isnull()
- notnull()
- dropna()
- fillna()

Laten we elk ervan bekijken en zien hoe ze werken, eerst in de context van `Series` objecten:

##### isnull()

Dit is een universele functie die wordt toegepast op elk element van de reeks:

In [3]:
s = pd.Series(['aaa', 'bbb', None, 'ddd', np.nan], index=list('abcde'))
s

a     aaa
b     bbb
c    None
d     ddd
e     NaN
dtype: object

In [4]:
pd.isnull(s)

a    False
b    False
c     True
d    False
e     True
dtype: bool

Zoals je kan zien, geeft dit een reeks van het type boolean terug (we zouden deze kunnen gebruiken voor boolean masking):

In [26]:
s[pd.isnull(s)]

c    None
e     NaN
dtype: object

Hiermee konden we alle `NaN` en `None` waarden extraheren.

##### notnull()

De `isnull()`-functie creëert in feite een masker waar de ontbrekende waarden `True` retourneren - we kunnen dit masker omkeren door `not` (`~`) te gebruiken:

In [27]:
s[~pd.isnull(s)]

a    aaa
b    bbb
d    ddd
dtype: object

Of, we kunnen gewoon de `notnull()` functie gebruiken:

In [28]:
s[pd.notnull(s)]

a    aaa
b    bbb
d    ddd
dtype: object

##### dropna()

Deze functie zal eventuele ontbrekende waarden uit de serie verwijderen:

In [29]:
s.dropna()

a    aaa
b    bbb
d    ddd
dtype: object

En opnieuw heeft dit geen invloed op de oorspronkelijke reeks, maar retourneert in plaats daarvan een nieuwe reeks, waarbij zoals gebruikelijk de indexlabels worden behouden voor de nieuwe reeks.

##### fillna()

Deze functie kan worden gebruikt om ontbrekende waarden in een reeks te vervangen door een andere waarde:

In [5]:
s

a     aaa
b     bbb
c    None
d     ddd
e     NaN
dtype: object

In [6]:
s.fillna('missing')

a        aaa
b        bbb
c    missing
d        ddd
e    missing
dtype: object

Maar waar `fillna()` pas echt interessant wordt, is wanneer we andere waarden in dezelfde serie gebruiken om de ontbrekende waarde aan te vullen. Pandas biedt twee veelgebruikte methoden.

De eerste optie is om de voorgaande (niet-ontbrekende) waarde te gebruiken. Dit heet een **forward fill**:

In [8]:
s.ffill()

a    aaa
b    bbb
c    bbb
d    ddd
e    ddd
dtype: object

We kunnen ook terugvulling gebruiken, dat kijkt naar de volgende niet-ontbrekende waarde:

In [9]:
s.bfill()

a    aaa
b    bbb
c    ddd
d    ddd
e    NaN
dtype: object

Merk op hoe de laatste waarde niet werd vervangen - dat komt doordat er geen waarde was na de rij met `e`. En iets gelijkaardigs gebeurt bij voorwaarts vullen als de eerste waarde ontbreekt.

Je kunt zowel een achterwaartse vulling als een voorwaartse vulling gebruiken om de uiteinden te behandelen:

In [10]:
s.ffill().bfill()

a    aaa
b    bbb
c    bbb
d    ddd
e    ddd
dtype: object

Gerelateerd aan deze `fillna()` functie is een meer geavanceerde functie genaamd `interpolate()` die kan worden gebruikt voor meer geavanceerde technieken voor het opvullen van ontbrekende waarden. We zullen dit niet in detail bestuderen - maar je kan meer info terugvinden op
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html

In [12]:
s = pd.Series([1, 2, None, 4, None, 7])
s

0    1.0
1    2.0
2    NaN
3    4.0
4    NaN
5    7.0
dtype: float64

In [13]:
s.interpolate(method='linear')

0    1.0
1    2.0
2    3.0
3    4.0
4    5.5
5    7.0
dtype: float64

De lineaire methode gaat er in feite vanuit dat de getallen gelijkmatig verdeeld zijn en vult ontbrekende waarden op die manier in.

#### Werken met Ontbrekende Waarden in DataFrames

Nu bekijken we hoe dit alles werkt in de context van DataFrames:

In [14]:
d = {
    'col1': {'row1': 1, 'row2': 10, 'row3': 100, 'row4': 1000, 'row5': 10000},
    'col2': {'row1': 2, 'row2': None, 'row3': None, 'row4': 2000, 'row5': 20000},
    'col3': {'row1': 3, 'row2': 30, 'row3': 300, 'row4': None, 'row5': 40000},
    'col4': {'row1': 4, 'row2': 40, 'row3': 400, 'row4': 4000, 'row5': 40000}
}

df = pd.DataFrame(d)
df

,col1,col2,col3,col4
row1,1,2.0,3.0,4
row2,10,NaN,30.0,40
row3,100,NaN,300.0,400
row4,1000,2000.0,NaN,4000
row5,10000,20000.0,40000.0,40000


We kunnen de `isnull()` en `notnull()` functies op het volledige dataframe gebruiken:

In [15]:
df.isnull()

,col1,col2,col3,col4
row1,False,False,False,False
row2,False,True,False,False
row3,False,True,False,False
row4,False,False,True,False
row5,False,False,False,False


Zoals je kunt zien krijgen we een masker voor elk element van de matrix.

De `fillna()` kan ook toegepast worden op het volledige data frame:

In [39]:
df.fillna(0)

,col1,col2,col3,col4
row1,1,2.0,3.0,4
row2,10,0.0,30.0,40
row3,100,0.0,300.0,400
row4,1000,2000.0,0.0,4000
row5,10000,20000.0,40000.0,40000


We kunnen ook een methode voor achterwaarts/voorwaarts invullen gebruiken - maar vul je dan in op basis van de kolomwaarden of de rijwaarden?

Laten we het proberen:

In [16]:
print(df)
df.fillna(method='ffill')

       col1     col2     col3   col4
row1      1      2.0      3.0      4
row2     10      NaN     30.0     40
row3    100      NaN    300.0    400
row4   1000   2000.0      NaN   4000
row5  10000  20000.0  40000.0  40000


/tmp/ipykernel_2715/400473987.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill')


,col1,col2,col3,col4
row1,1,2.0,3.0,4
row2,10,2.0,30.0,40
row3,100,2.0,300.0,400
row4,1000,2000.0,300.0,4000
row5,10000,20000.0,40000.0,40000


Zoals je kunt zien, maakte dit gebruik van een **forward fill gebaseerd op de kolomwaarden**.

Maar we willen mogelijk waarden invullen op basis van de rijen, niet de kolommen.
We kunnen dit specifiëren door middel van het `axis` argument :

In [17]:
print(df)
df.fillna(method='ffill', axis=1)

       col1     col2     col3   col4
row1      1      2.0      3.0      4
row2     10      NaN     30.0     40
row3    100      NaN    300.0    400
row4   1000   2000.0      NaN   4000
row5  10000  20000.0  40000.0  40000


/tmp/ipykernel_2715/1604425169.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=1)


,col1,col2,col3,col4
row1,1.0,2.0,3.0,4.0
row2,10.0,10.0,30.0,40.0
row3,100.0,100.0,300.0,400.0
row4,1000.0,2000.0,2000.0,4000.0
row5,10000.0,20000.0,40000.0,40000.0


De backfill-methode werkt op dezelfde manier.

De `interpolate` functie werkt ook op DataFrames, en net als de `fillna` methode, kunnen we ook de as specificeren 'waarlangs' we willen vullen.

In [18]:
print(df)
df.interpolate(method='linear')

       col1     col2     col3   col4
row1      1      2.0      3.0      4
row2     10      NaN     30.0     40
row3    100      NaN    300.0    400
row4   1000   2000.0      NaN   4000
row5  10000  20000.0  40000.0  40000


,col1,col2,col3,col4
row1,1,2.0,3.0,4
row2,10,668.0,30.0,40
row3,100,1334.0,300.0,400
row4,1000,2000.0,20150.0,4000
row5,10000,20000.0,40000.0,40000


We kunnen ook interpoleren langs de kolomas:

In [19]:
print(df)
df.interpolate(method='linear', axis=1)

       col1     col2     col3   col4
row1      1      2.0      3.0      4
row2     10      NaN     30.0     40
row3    100      NaN    300.0    400
row4   1000   2000.0      NaN   4000
row5  10000  20000.0  40000.0  40000


,col1,col2,col3,col4
row1,1.0,2.0,3.0,4.0
row2,10.0,20.0,30.0,40.0
row3,100.0,200.0,300.0,400.0
row4,1000.0,2000.0,3000.0,4000.0
row5,10000.0,20000.0,40000.0,40000.0


Als laatste hebben we de `dropna()` methode.
Dit kan worden gebruikt om rijen of kolommen te droppen (laten vallen) die null waarden bevatten.  We moeten specifiëren langs welke as we dit wensen te doen (`0` om rijen te droppen, `1` om kolommen te droppen, met `0` als default value):

In [23]:
print(df)
df.dropna(subset=['col2'])

       col1     col2     col3   col4
row1      1      2.0      3.0      4
row2     10      NaN     30.0     40
row3    100      NaN    300.0    400
row4   1000   2000.0      NaN   4000
row5  10000  20000.0  40000.0  40000


,col1,col2,col3,col4
row1,1,2.0,3.0,4
row4,1000,2000.0,NaN,4000
row5,10000,20000.0,40000.0,40000


Zoals je kunt zien, zijn alle **rijen** verwijderd die een null-waarde bevatten.

Aan de andere kant willen we mogelijk **kolommen** verwijderen die null-waarden bevatten:

In [21]:
print(df)
df.dropna(axis=1)

       col1     col2     col3   col4
row1      1      2.0      3.0      4
row2     10      NaN     30.0     40
row3    100      NaN    300.0    400
row4   1000   2000.0      NaN   4000
row5  10000  20000.0  40000.0  40000


,col1,col4
row1,1,4
row2,10,40
row3,100,400
row4,1000,4000
row5,10000,40000
